In [9]:
import subprocess
import time
import multiprocessing
#from concurrent.futures import ThreadPoolExecutor

worker="ssh -t fhl@localhost "
master="ssh -t fhl@ict.acs.edu "

commandWorkload=master + "\"sudo docker exec redis-client-1 memtier_benchmark -s envoy {}\""
commandStressCPU=worker + "\"sudo podman run -d --rm --cpuset-cpus 131 my-stress-ng:v0.1 {}\""
commandStressMem=worker + "\"sudo podman run -d --rm --cpuset-cpus 40-50,52-59,120-130,132-139 my-stress-ng:v0.1 {}\""

commandStress = commandStressMem



# paras
perTestDuration = 900 #second
workloadThreads = [5,10,15,20]
stressCPUworker=[10,20,30,40,50,60]
stressMemWorker=[1,2,3,4,5,6]
stressIOWorker=[1,2,4,6]
sleepGap = 60

data_dir="./data/{}"

# paras template
workload="--test-time {} -t {}"
stressCPU="--cpu 1 --cpu-load {} "
stressMem="--memrate {} --memrate-bytes 1G"
stressIO="--io {} "
stressTime="--timeout {}s"

# init commands
one_limit=6
two_limit=2
three_limit=1
# mode = [("CPU",stressCPU,stressCPUworker),("Mem",stressMem,stressMemWorker),("IO",stressIO,stressIOWorker)]
mode = [("Mem",stressMem,stressMemWorker)]

commandsDict={
    "step0":{},
    "step1":{},
    "step2":{},
    "step3":{}
}
resultsFile={
    "step0":{},
    "step1":{},
    "step2":{},
    "step3":{}
}
exect = 0
for n in workloadThreads:
    commands = []
    files = []
    
    tag = "exec{}-load{}".format(exect,n)
    
    commands.append(commandWorkload.format(workload.format(perTestDuration,n)))
    files.append("workload-{}".format(tag))
    
    commandsDict["step0"][tag]=commands.copy()
    resultsFile["step0"][tag]=files.copy()
    
    exect = exect + 1

for n in workloadThreads:
    for label,template,parasList in mode:
        for para in parasList[:one_limit]:
            commands = []
            files = []
            
            tag = "exec{}-load{}-{}-{}".format(exect,n,label,para)
            commands.append(commandWorkload.format(workload.format(perTestDuration,n)))
            files.append("workload-{}".format(tag))
            
            commands.append(commandStress.format(template.format(para)+stressTime.format(perTestDuration)))
            files.append("stress-{}".format(tag))
            
            commandsDict["step1"][tag]=commands.copy()
            resultsFile["step1"][tag]=files.copy()
            exect = exect + 1

# for n in workloadThreads:
#     for id1, (l1, t1, paras1List) in enumerate(mode):
#         for id2, (l2, t2, paras2List) in enumerate(mode):
#             if id2 > id1:
#                 for p1 in paras1List[:two_limit]:
#                     for p2 in paras2List[:two_limit]:
#                         commands = []
#                         files = []
                        
#                         tag = "exec{}-load{}-{}-{}-{}-{}".format(exect,n,l1,p1,l2,p2)
#                         commands.append(commandWorkload.format(workload.format(perTestDuration,n)))
#                         files.append("workload-{}".format(tag))
#                         commands.append(commandStress.format(t1.format(p1) + t2.format(p2) + stressTime.format(perTestDuration)))
#                         files.append("stress-{}".format(tag))
                        
#                         commandsDict["step2"][tag]=commands.copy()
#                         resultsFile["step2"][tag]=files.copy()
#                         exect = exect + 1

# for n in workloadThreads:
#     for id1, (l1, t1, paras1List) in enumerate(mode):
#         for id2, (l2, t2, paras2List) in enumerate(mode):
#             for id3, (l3, t3, paras3List) in enumerate(mode):
#                 if id2 > id1 and id3 > id2:
#                     for p1 in paras1List[:three_limit]:
#                         for p2 in paras2List[:three_limit]:
#                             for p3 in paras3List[:three_limit]:
#                                 commands = []
#                                 files = []
                                
#                                 tag = "exec{}-load{}-{}-{}-{}-{}-{}-{}".format(exect,n,l1,p1,l2,p2,l3,p3)
#                                 commands.append(commandWorkload.format(workload.format(perTestDuration,n)))
#                                 files.append("workload-{}".format(tag))
#                                 commands.append(commandStress.format(t1.format(p1) + t2.format(p2) + stressTime.format(perTestDuration)))
#                                 files.append("stress-{}".format(tag))
                                
#                                 commandsDict["step3"][tag]=commands.copy()
#                                 resultsFile["step3"][tag]=files.copy()
#                                 exect = exect + 1

# exec commands
def run_command(para):
    (command,file) = para
    print("\nstart time {} file: {} excute commands: {}".format(time.time(),data_dir.format(file),command))

    output_file = open(data_dir.format(file), "w")
    result = subprocess.run(command, shell=True, stdout=output_file, text=True)
    output_file.close()

    print("\nend time {} file: {} excute commands: {}".format(time.time(),data_dir.format(file),command))
    
    
for step,cmds in commandsDict.items():
    for label,cmds in cmds.items():
        futures = []

        for id,cmd in enumerate(cmds):
            futures.append(multiprocessing.Process(target=run_command, args=((cmd, resultsFile[step][label][id]),)))
        for p in futures:
            p.start()
        for p in futures:
            p.join()
            
        # with ThreadPoolExecutor() as executor:
        #     for id,cmd in enumerate(cmds):
        #         futures.append(executor.submit(run_command, (cmd, resultsFile[step][label][id])))
                
        # for future in futures:
        #    future.result()
        # time.sleep(sleepGap)


start time 1691638035.8563557 file: ./data/workload-exec0-load5 excute commands: ssh -t fhl@ict.acs.edu "sudo docker exec redis-client-1 memtier_benchmark -s envoy --test-time 900 -t 5"

start time 1691638035.876612 file: ./data/workload-exec1-load10 excute commands: ssh -t fhl@ict.acs.edu "sudo docker exec redis-client-1 memtier_benchmark -s envoy --test-time 900 -t 10"

start time 1691638035.9294908 file: ./data/workload-exec2-load15 excute commands: ssh -t fhl@ict.acs.edu "sudo docker exec redis-client-1 memtier_benchmark -s envoy --test-time 900 -t 15"

start time 1691638035.9806018 file: ./data/workload-exec3-load20 excute commands: ssh -t fhl@ict.acs.edu "sudo docker exec redis-client-1 memtier_benchmark -s envoy --test-time 900 -t 20"

start time 1691638036.0270822 file: ./data/stress-exec4-load5-Mem-1 excute commands: ssh -t fhl@localhost "sudo podman run -d --rm --cpuset-cpus 40-50,52-59,120-130,132-139 my-stress-ng:v0.1 --memrate 1 --memrate-bytes 1G--timeout 900s"

start ti